In [26]:
import pandas as pd
import numpy as np

In [27]:
station = pd.read_csv('data/station.csv', usecols=['id','city'])
station.head(3)

,id,city
0,2,San Jose
1,3,San Jose
2,4,San Jose


In [28]:
trip = pd.read_csv('data/trip_train.csv',
                   usecols=['duration', 'start_date', 'start_station_id', 'subscription_type'])
trip.head(3)

,duration,start_date,start_station_id,subscription_type
0,396,8/27/2015 8:36,50,Subscriber
1,636,7/28/2014 22:06,67,Subscriber
2,334,6/9/2014 8:42,77,Subscriber


In [29]:
weather = pd.read_csv('data/weather.csv')

In [39]:
def preprocess(station, weather, trip):
    station = preprocessStation(station)
    weather = preprocessWeather(weather)
    trip = preprocessTrip(trip)
    trip = trip.merge(station, left_on='start_station_id', right_on='id')
    trip.drop('id', axis=1, inplace=True)
    trip = trip.merge(weather, left_on=['start_date', 'zip_code'], right_on=['date', 'zip_code'], how='outer')
    trip.drop(['date', 'zip_code'], axis=1, inplace=True)
    return encode(trip)
    
def preprocessStation(df):
    df = df.copy()
    df["zip_code"] = df.city.apply(zipCode)
    df.drop(labels=['city'], axis=1, inplace=True)
    return df

def preprocessTrip(df):
    df = df.copy()
    df['start_date'] = df['start_date'].apply(lambda x : pd.to_datetime(x))
    df['time'] = df.start_date.apply(lambda x: 60 * x.hour + x.minute)
    df['year'] = df.start_date.apply(lambda x : x.year)
    df['month'] = df.start_date.apply(lambda x : x.month)
    df['day'] = df.start_date.apply(lambda x : x.day)
    df['dayofweek'] = df.start_date.apply(lambda x : x.isoweekday())
    df['subscription_type'] = df.subscription_type.apply(lambda x: 1 if x == "Subscriber" else 0)
#    df = df.merge(holidays.to_frame(), left_on='start_date', right_on=holidays.to_frame().index, how='outer')
    return df

def preprocessWeather(df):
    df = df.copy()
    cleanPrecipitation(df)
    cleanEvents(df)
    df = df.fillna(0)
    df['date'] = df.date.apply(lambda x: pd.to_datetime(x).date())
    numberEvents(df)
    return df

def numberEvents(df):
    df['events'] = df.events.apply(eventNumber)

def eventNumber(eventString):
    if eventString == 'Normal': return 0
    elif eventString == 'Rain': return 1
    elif eventString == 'Fog': return 2
    elif eventString == 'Fog-Rain': return 3
    elif eventString == 'Rain-Thunderstom ': return 4

def zipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

def cleanPrecipitation(df):
    df.dropna(subset=["precipitation_inches"], inplace=True)
    df['precipitation_inches'] = df.precipitation_inches.apply(lambda x:
                                                               0.005 if (x == 'T')
                                                               else float(x))
def cleanEvents(df):
    df['events'] = df.events.apply(lambda x: "Normal" if pd.isnull(x) else x)
    df['events'] = df.events.apply(lambda x: "Rain" if x == "rain" else x)

def encode(df):
    encoder = sklearn.preprocessing.OneHotEncoder(categorical_features=(2, 3, 5, 6, 7, 8, 29))
    return pd.DataFrame(encoder.fit_transform(df).toarray())

from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH
cal = calendarUSFH()
holidays = cal.holidays(return_name=True, start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))

import sklearn.preprocessing

In [38]:
df = preprocess(station, weather, trip)
df.head(5)

ValueError: could not convert string to float: Subscriber